In [1]:
import os
import ogr
import shapefile as shp
from math import ceil
# from collections import Counter
from shapely.geometry import Point
from shapely.geometry import Polygon
from rtree import index
import pandas as pd
from datetime import datetime

startTime = datetime.now() #00:00:56.429351

In [2]:
#user inputs
# outPath = "/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/" #folder to dump results
# pointfeat = '/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/ChinaVIIRS.shp' #starting points
outPath = "/Users/itclunie/Desktop/ECON/find industry/speedTest/" #folder to dump results
pointfeat = '/Users/itclunie/Desktop/ECON/find industry/speedTest/VIIRSclustRd.shp' #starting points

polyfeat = outPath + 'outGrid2.shp' #name your output grid

gridCutoff = 10 #gridcell has to have n num of months with values to be flagged
gridHeight = .015 #.015 = 1.5km
gridWidth = .015

In [3]:
def makeGrid(outputGridfn,xmin,xmax,ymin,ymax,gridHeight,gridWidth):
    # convert sys.argv to float
    xmin = float(xmin)
    xmax = float(xmax)
    ymin = float(ymin)
    ymax = float(ymax)
    gridWidth = float(gridWidth)
    gridHeight = float(gridHeight)

    # get rows
    rows = ceil((ymax-ymin)/gridHeight)
    # get columns
    cols = ceil((xmax-xmin)/gridWidth)

    # start grid cell envelope
    ringXleftOrigin = xmin
    ringXrightOrigin = xmin + gridWidth
    ringYtopOrigin = ymax
    ringYbottomOrigin = ymax-gridHeight

    # create output file
    outDriver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputGridfn):
        os.remove(outputGridfn)
        
    outDataSource = outDriver.CreateDataSource(outputGridfn)
    outLayer = outDataSource.CreateLayer(outputGridfn,geom_type=ogr.wkbPolygon )
    featureDefn = outLayer.GetLayerDefn()

    # create grid cells
    countcols = 0
    while countcols < cols:
        countcols += 1

        # reset envelope for rows
        ringYtop = ringYtopOrigin
        ringYbottom =ringYbottomOrigin
        countrows = 0

        while countrows < rows:
            countrows += 1
            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(ringXleftOrigin, ringYtop)
            ring.AddPoint(ringXrightOrigin, ringYtop)
            ring.AddPoint(ringXrightOrigin, ringYbottom)
            ring.AddPoint(ringXleftOrigin, ringYbottom)
            ring.AddPoint(ringXleftOrigin, ringYtop)
            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)

            # add new geom to layer
            outFeature = ogr.Feature(featureDefn)
            outFeature.SetGeometry(poly)
            outLayer.CreateFeature(outFeature)
            outFeature.Destroy

            # new envelope for next poly
            ringYtop = ringYtop - gridHeight
            ringYbottom = ringYbottom - gridHeight

        # new envelope for next poly
        ringXleftOrigin = ringXleftOrigin + gridWidth
        ringXrightOrigin = ringXrightOrigin + gridWidth

    # Close DataSources
    outDataSource.Destroy()

In [4]:
#make grid
pointfeatExtent = shp.Reader(pointfeat)

xmin = pointfeatExtent.bbox[0]
xmax = pointfeatExtent.bbox[2]
ymin = pointfeatExtent.bbox[1]
ymax = pointfeatExtent.bbox[3]

makeGrid(polyfeat,xmin,xmax,ymin,ymax,gridHeight,gridWidth)

In [5]:
#A Load the shapefile of polygons and convert it to shapely polygon objects
polygons_sf = shp.Reader(polyfeat)
polygon_shapes = polygons_sf.shapes()
polygon_points = [q.points for q in polygon_shapes ]
polygons = [Polygon(q) for q in polygon_points]
poly_records = polygons_sf.records()

In [6]:
#B Load the shapefile of points and convert it to shapely point objects
points_sf = shp.Reader(pointfeat)
pntRecords = points_sf.shapeRecords()
point_coords = [q.shape.points[0] for q in pntRecords ]
points = [Point(q.shape.points[0]) for q in pntRecords ]

In [7]:
#C structure tally dictionaries
GRIDdict = {}
subGRIDdict = {}
VIIRSdict = {}
for i in pntRecords:
    VIIRSdict[i.record[4]] = [] #add keys (monthYear) to VIIRSdict & GRIDdict
    GRIDdict[i.record[4]] = []
    
# for i in pntRecords: #fill VIIRSdict.  
#     VIIRSdict[i.record[4]].append(i) #key: monthYear, value: shape and attributes
[VIIRSdict[i.record[4]].append(i) for i in pntRecords]

for i in poly_records: #fill subGRIDdict.  
    subGRIDdict[i[0]] = None ##key= GRIDID, value= {GRIDid:0...    looks like {0:0, 1:0, 2:0 ...

for key in GRIDdict:  #fill GRIDdict. 
    GRIDdict[key] = dict(subGRIDdict)  #key= 8_2012  value=  copies of subGRIDdict  
    

In [8]:
#D Build a spatial index based on the bounding boxes of the polygons
idx = index.Index()

[ idx.insert(i, polygon_shapes[i].bbox) for i in range(len(polygon_shapes)) ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [32]:
#actual points in grid cells matching


# t_1 = []
# t_2 = []
# t_3 = []
countr=0
for key in VIIRSdict:
    tallyHO = []
    point_coords = [ q.shape.points[0] for q in VIIRSdict[key] ]
    points = [ Point(q.shape.points[0]) for q in VIIRSdict[key] ]    
    
    pc = 0
#     t1 = datetime.now()
    for i in range(len(VIIRSdict[key])): #Iterate through each point
        pc+=1
        if pc % 1000 == 0:
            print('-----',pc)
            
        for j in idx.intersection(point_coords[i]):
            if points[i].within(polygons[j]):       
                tallyHO.append(poly_records[j][0]) 
                break 
                
#     t1 = datetime.now() - t1
#     t_1.append(t1.microseconds)
    
    
#     t2 = datetime.now()
    resultDict = dict([ (i,tallyHO.count(i)) for i in set(tallyHO) ])
    t2 = datetime.now() - t2
#     t_2.append(t2.microseconds)
    
    
#     t3 = datetime.now()
    for rkey in resultDict:
        GRIDdict[key][rkey] = resultDict[rkey] 
#     t3 = datetime.now() - t3
#     t_3.append(t3.microseconds)
    
    countr += 1
    print( countr, len( VIIRSdict.keys() ) )

# print(sum(t_1) / len(t_1))   #57330.958333333336
# print(sum(t_2) / len(t_2))   #14091.416666666666
# print(sum(t_3) / len(t_3))   #129.40277777777777


----- 1000
1 72
----- 1000
2 72
----- 1000
----- 2000
3 72
4 72
5 72
6 72
----- 1000
7 72
----- 1000
----- 2000
----- 3000
8 72
----- 1000
9 72
----- 1000
----- 2000
----- 3000
10 72
11 72
12 72
13 72
----- 1000
14 72
----- 1000
15 72
----- 1000
----- 2000
----- 3000
16 72
----- 1000
----- 2000
----- 3000
17 72
----- 1000
----- 2000
18 72
----- 1000
----- 2000
----- 3000
19 72
----- 1000
----- 2000
----- 3000
----- 4000
20 72
21 72
22 72
23 72
24 72
----- 1000
25 72
----- 1000
26 72
----- 1000
27 72
28 72
29 72
30 72
31 72
32 72
----- 1000
33 72
----- 1000
----- 2000
34 72
----- 1000
35 72
----- 1000
36 72
----- 1000
----- 2000
37 72
38 72
39 72
40 72
41 72
42 72
----- 1000
43 72
44 72
45 72
----- 1000
46 72
47 72
----- 1000
48 72
49 72
----- 1000
50 72
51 72
----- 1000
----- 2000
52 72
53 72
54 72
55 72
56 72
57 72
58 72
59 72
60 72
61 72
62 72
63 72
64 72
65 72
66 72
67 72
68 72
----- 1000
----- 2000
69 72
----- 1000
70 72
71 72
72 72


In [30]:
#remove empty rows (ie empty grid squares), pick out hexes of interest, add centroids
dfClean = pd.DataFrame.from_dict(GRIDdict, orient='columns', dtype=None)
dfClean['OfInterest'] = None
dfClean['X'] = None
dfClean['Y'] = None

dropLst = []
for i in range(len(dfClean)):
    t3 = datetime.now()
        
    row = dfClean.iloc[i]    
    
    if row.sum() >= gridCutoff:  #user input
        dfClean.at[i,'OfInterest'] = row.sum()
        
        centroid = polygons[i].centroid
        cX = centroid.coords[0][0]
        cY = centroid.coords[0][1]
        dfClean.at[i,'X'] = cX
        dfClean.at[i,'Y'] = cY
    else:
        dropLst.append(i)
        
    t3 = datetime.now() - t3
    t_3.append(t3.microseconds)
 
print(sum(t_3) / len(t_3))
#df = dfClean.drop(dropLst)

311.6503436865715


In [ ]:

# keepers2 = []
# for i in range(len(dfClean)):
#     row = dfClean.iloc[i]   
#     rowCount = []
#     [rowCount.append(1) for j in range(len(row)) if "_" in row.index[j] and row[j] > 0]

#     [keepers2.append(i) if sum(rowCount) >= 10]
            
# print(keepers2)

In [ ]:


keepers2 = []
for i in range(len(dfClean)):
    row = dfClean.iloc[i] 
    
    print(row)
    print(row.count())
    
#     for j in range(len(row[:-3])): #everything except last 3 columns, x y ofinterest
        
#         print(row.index[j], row[j])
    break

In [ ]:
#write output files

df.to_csv(outPath + 'industryFindR_wide.csv')
dfClean['GRIDid'] = dfClean.index
melted = dfClean.melt(id_vars=['GRIDid','OfInterest','X','Y']) 
melted.to_csv(outPath + 'industryFindR_long.csv')

print(datetime.now() - startTime)